In [48]:
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# -*- coding: utf-8 -*-
import pandas as pd, numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ChromeOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from datetime import timedelta
import pathlib
import math
import time

Compute articles related to Luzern and save them in a temporary csv file

In [49]:
# get the date of the moste recent article in the merged dataset
# liverss_and_coms_path = pathlib.Path.home() / 'Desktop' / 'SocialMediaMonitor' / 'data' / 'raw' / 'liverss_and_coms_srf.csv'
# liverss_and_coms_df = pd.read_csv(liverss_and_coms_path)
# liverss_and_coms_df['published'] = pd.to_datetime(liverss_and_coms_df['published'])
# most_recent_article = liverss_and_coms_df['published'].max()

# get the ids of the articles that are less than 15 days old
# recent_date = pd.to_datetime(most_recent_article-timedelta(15), utc=True)
# mask = (liverss_and_coms_df['published'] > recent_date) # & (total_df['is_com_or_sub'] == 'sub')
# recent_articles = liverss_and_coms_df.loc[mask]
# recent_articles_id = recent_articles['id'].tolist()
# #print(recent_articles_id[:3])
# print(len(recent_articles_id))

In [50]:
# Configure the Chrome webdriver
driver = webdriver.Chrome('/users/michaellappert/Downloads/chromedriver') # Needs to be adjusted client independently

# get the articles of srf related to Luzern
driver.get('https://www.srf.ch/suche?q=Luzern&date=all&page=0')

for _ in range(1): # change this number if more articles need to be loaded but 2 pages should be sufficient
    try:
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME, "button.button--white.button--mixed-case.button--link-text-color.button--no-spacing.show-more-bar__child.show-more-bar__child--grow.js-show-more-button"))).click()
        print("LOAD MORE ARTICLES button clicked")
    except TimeoutException:
        print("No more LOAD MORE ARTICLES button to be clicked")
        break



LOAD MORE ARTICLES button clicked


In [53]:
# get headers of the articles
headers = driver.find_elements_by_class_name('teaser__kicker-text')
headers_list = [header.get_attribute('innerHTML') for header in headers]
headers_list = headers_list[3:]
# print(headers_list)

In [54]:
# get the titles of the articles
titles = driver.find_elements_by_class_name('teaser__title')
titles_list = [title.get_attribute('innerHTML') for title in titles]
# print('1. print statement:')
# print(titles_list)
titles_list = titles_list[10:]
# print('2. print statement:')
# print(titles_list)

In [55]:
# get the summaries of the articles
summaries = driver.find_elements_by_class_name('teaser__lead.teaser__lead--search')
summaries_list = [summary.get_attribute('innerHTML') for summary in summaries]
summaries_list = [summary.replace('<em>', '').replace('</em>', '').replace('<span>', '').replace('</span>', '') for summary in summaries_list]
# print(summaries_list)

In [56]:
# get the links of the articles
links = driver.find_elements_by_class_name('teaser__main.teaser__main--search')
links_list = [link.get_attribute('href') for link in links]
# print(links_list)

In [57]:
dates = driver.find_elements_by_class_name('teaser-info.teaser-info--with-updates.teaser-info--uppercase')
dates_list = [date.get_attribute('innerHTML') for date in dates]
dates_list_formatted = []
# print(dates_list)
for date in dates_list:
    try:
        curr_date = pd.to_datetime(date, format='%d.%m.%Y, %H:%M Uhr', utc=True)
    except ValueError:
        print(f'Date triggered ValueError: {date}')
        curr_date = str(date)[0:12] + '00:' + str(date)[15:17]
        curr_date = pd.to_datetime(curr_date, format='%d.%m.%Y, %H:%M', utc=True)
    dates_list_formatted.append(curr_date)

print(dates_list_formatted[:3])
driver.quit()

[Timestamp('2022-05-08 18:35:00+0000', tz='UTC'), Timestamp('2022-05-01 18:40:00+0000', tz='UTC'), Timestamp('2022-04-26 16:39:00+0000', tz='UTC')]


In [58]:
articles = pd.DataFrame(list(zip(headers_list, titles_list, dates_list_formatted, summaries_list, 
                                 links_list, links_list)), 
                        columns =['header', 'title', 'published', 'summary', 'link', 'id'])
articles['is_com_or_sub'] = 'sub'

In [59]:
# Compute a Pandas dataframe to write into srf articles csv file
liverss_df = articles # articles[articles['date'] > last_sub_check] -> last_sub_check not done yet
liverss_df.head()


,header,title,published,summary,link,id,is_com_or_sub
0,4:0-Heimsieg gegen Servette,Luzern mindestens in der Barrage und nur noch ...,2022-05-08 18:35:00+00:00,Luzern gewinnt in der 33. SL-Runde gegen Serve...,https://www.srf.ch/sport/fussball/super-league...,https://www.srf.ch/sport/fussball/super-league...,sub
1,3:0-Heimsieg gegen Lausanne,Luzern feiert wichtigen Erfolg im Abstiegskampf,2022-05-01 18:40:00+00:00,Der FC Luzern feiert im Kellerduell der 32. Ru...,https://www.srf.ch/sport/fussball/super-league...,https://www.srf.ch/sport/fussball/super-league...,sub
2,Abstimmung Stadt Luzern,Braucht Luzern mehr zusammenhängende Velorouten?,2022-04-26 16:39:00+00:00,… unsichere Stellen für Velofahrende in der St...,https://www.srf.ch/news/abstimmungen-15-mai-20...,https://www.srf.ch/news/abstimmungen-15-mai-20...,sub
3,Mitten in den Playoffs,Perkovac bei Kriens-Luzern per sofort freigest...,2022-04-26 14:45:00+00:00,Kriens-Luzern trennt sich per sofort und nicht...,https://www.srf.ch/sport/mehr-sport/handball/m...,https://www.srf.ch/sport/mehr-sport/handball/m...,sub
4,Basel schlägt Luzern mit 3:0,Spielverderber Stocker: FCZ-Meisterparty muss ...,2022-04-24 18:37:00+00:00,Basel schlägt Luzern in der 31. Runde der Supe...,https://www.srf.ch/sport/fussball/super-league...,https://www.srf.ch/sport/fussball/super-league...,sub


In [60]:
# Saving generated data to raw data folder
liverss_df.to_csv(r'~/Desktop/SocialMediaMonitor/data/raw/liverss_srf.csv', index = False)